In [1]:
import numpy as np

from src.models.train_emos import train_emos, train_and_test_emos
from pit import make_cpit_diagram_emos, make_cpit_hist_emos 
from brier_score import brier_skill_plot, brier_plot
from src.models.get_data import get_tensors, get_normalized_tensor
from src.models.emos import EMOS

2024-03-01 09:21:13.461849: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 09:21:13.488177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 09:21:13.488196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 09:21:13.488978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 09:21:13.493193: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 09:21:13.493678: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
neighbourhood_size = 11
parameter_names = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']
ignore = ['229', '285', '323']
train_folds = [1, 2]
train_data = get_normalized_tensor(neighbourhood_size, parameter_names, train_folds, ignore)

X_train = train_data['X']
y_train = train_data['y']
variances_train = train_data['variances']
mean_train = train_data['mean']
std_train = train_data['std']


print(X_train.shape)

(15733, 5)


In [3]:
test_fold = 3

X_test, y_test, variances_test = get_tensors(neighbourhood_size, parameter_names, test_fold, ignore)
X_test = (X_test - mean_train) / std_train

print(X_test.shape)

(7780, 5)


In [4]:
setup = {}

setup["num_features"] = len(parameter_names)
setup["feature_mean"] = mean_train
setup["feature_std"] = std_train
setup["features"] = parameter_names
setup["neighbourhood_size"] = neighbourhood_size

In [7]:
setup1 = setup

setup1["loss"] = "loss_CRPS_sample"
setup1["samples"] = 100
setup1["optimizer"] = "Adam"
setup1["learning_rate"] = 0.01
setup1["forecast_distribution"] = "distr_trunc_normal"

trunc_normal_crps = EMOS(setup1)

Using default parameters for truncated normal distribution


{'num_features': 5, 'feature_mean': <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=
array([0.0000000e+00, 1.0172716e+05, 1.4279445e-01, 8.9635211e-01,
       2.9374678e+04], dtype=float32)>, 'feature_std': <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=
array([1.0000000e+00, 1.1295465e+03, 3.5499525e-01, 8.0064192e-02,
       1.0786042e+03], dtype=float32)>, 'features': ['wind_speed', 'press', 'kinetic', 'humid', 'geopot'], 'neighbourhood_size': 11, 'loss': 'loss_CRPS_sample', 'samples': 100, 'optimizer': 'Adam', 'learning_rate': 0.01, 'forecast_distribution': 'distr_trunc_normal'}
